In [ ]:
!rm -rf *
!mkdir /kaggle/working/stems
!mkdir /kaggle/working/midis
!mkdir /kaggle/working/synths

In [ ]:
!pip install spleeter --quiet 
!pip install basic-pitch --quiet 
!pip install soundfile --quiet 
!pip install midi2audio --quiet 
!pip install pydub --quiet 
!pip install pysndfx --quiet 
!apt-get install -qq sox fluidsynth

In [ ]:
import random
import os
import sys
import subprocess as proc 
import time
import random

import soundfile as sf
import librosa as librosa
from pydub import AudioSegment

print(sys.path)
now = time.time()

In [ ]:
now_spleeter = time.time()
input_dir = "/kaggle/input/orchestralai-input"
audio = os.path.join(input_dir,os.listdir(input_dir)[0])
print(audio)
cmd = "spleeter separate -p spleeter:4stems -o stems "+audio
proc.run(cmd.split(" "))
print(f"stems extracted")
now_spleeter = time.time()-now_spleeter

In [ ]:
now_basicp = time.time()
stems_dir = "/kaggle/working/stems/"+audio.split("/")[-1].split(".")[0]
stems = [os.path.join(stems_dir,f) for f in os.listdir(stems_dir)]
cmd = f"basic-pitch /kaggle/working/midis "+" ".join(stems)
proc.run(cmd.split(" "))

print("midis generation done")
now_basicp = time.time() - now_basicp

In [ ]:
now_synth = time.time()
soundfont_path = "/kaggle/input/sonanita-sf2"
soundfont_files = os.listdir(soundfont_path)

vocal_sf = [sf2 for sf2 in soundfont_files if "Violins Staccato" in sf2]
bass_sf = [sf2 for sf2 in soundfont_files if "Brass" in sf2 or "Basses" in sf2]
other_sf = [sf2 for sf2 in soundfont_files if "Woodwinds" in sf2]
drums_sf = [sf2 for sf2 in soundfont_files if "Keys" in sf2]

midi_files = [f for f in os.listdir("/kaggle/working/midis")]

synth_dir = "/kaggle/working/synths"
for midi in midi_files :
    sfz = ""
    if "vocals" in midi : sfz = os.path.join(soundfont_path,random.choice(vocal_sf))
    if "drums" in midi : sfz = os.path.join(soundfont_path,random.choice(bass_sf))
    if "bass" in midi : sfz = os.path.join(soundfont_path,random.choice(other_sf))
    if "other" in midi : sfz = os.path.join(soundfont_path,random.choice(drums_sf))
    print(sfz)
    midi_path = os.path.join("/kaggle/working/midis",midi)
    synth_path = os.path.join(synth_dir,midi.split("_")[0]+"_synths.wav")
    proc.run(["midi2audio","-s",sfz,midi_path,synth_path])
print("synths generated")
now_synth = time.time()-now_synth

In [ ]:
now_overlay = time.time()
synth_files = [f for f in os.listdir(synth_dir)]
overlay = None
for s in synth_files :
    audio_segment = AudioSegment.from_file(os.path.join(synth_dir,s), format="wav")
    if overlay :
        overlay = overlay.overlay(audio_segment,position=0)
    else :
        overlay = audio_segment
        
overlay.export("/kaggle/working/result.wav", format="wav")
from pysndfx import AudioEffectsChain
fx = (
    AudioEffectsChain()
    .reverb()
)
# Apply phaser and reverb directly to an audio file.
fx("/kaggle/working/result.wav", "/kaggle/working/result_reverb.wav")

print(f"spleeter ({now_spleeter} seconds)")
print(f"basic pitch ({now_basicp} seconds)")
print(f"synthesize ({now_synth} seconds)")
print(f"overlay ({time.time()-now_overlay} seconds)")
print(f"done converting ({time.time()-now} seconds)")

!rm -rf /kaggle/working/stems
!rm -rf /kaggle/working/midis
!rm -rf /kaggle/working/synths
!rm -rf /kaggle/working/pretrained_models
!rm /kaggle/working/result.wav